<a href="https://colab.research.google.com/github/adithya-prabhu-22/Natural_language_processing_from_scratch/blob/main/duplicate_question_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
'''
1) took the sample of 30000 rows drom entier dataset
2)applied the preprocesing technique and tokenisation to create word list
3)found the vector embedding for all question of 300 dim using word2vec
4)created new feature to train my model based on question length , natural form of question
 and vector form of question
5)applied standard scaler to standardize the numerical col
6)built a ANN as my model for thgis problem with relu as activation func in inner hidden layers
7)trained the model and predicted the result with accutacy of 75%
'''

In [1]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow
from tensorflow import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
import re
!pip install gensim
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [3]:
df = pd.read_csv('/content/train.csv')
df=df.sample(30000)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
63416,63416,110364,110365,How do I calculate the equivalent weights?,How do you calculate equivalent weights?,1
31741,31741,58499,58500,Taxes in India: Can I download a soft copy of ...,Can you derive your income tax file from a PAN...,0
33081,33081,60824,60825,What are some classic logic puzzles?,What are some logical thinking puzzles?,1
184379,184379,281719,281720,I am a fourth year student of BA LLB from Indi...,I am in my second year of my graduation with o...,0
110883,110883,44638,181709,Is there any evidence for reincarnation?,Is there any concrete evidence of reincarnation?,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 63416 to 300323
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30000 non-null  int64 
 1   qid1          30000 non-null  int64 
 2   qid2          30000 non-null  int64 
 3   question1     30000 non-null  object
 4   question2     30000 non-null  object
 5   is_duplicate  30000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 1.6+ MB


In [5]:
sampled_row = df.sample(1).iloc[0] # handling null questions
df.fillna(sampled_row, inplace=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 63416 to 300323
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            30000 non-null  int64 
 1   qid1          30000 non-null  int64 
 2   qid2          30000 non-null  int64 
 3   question1     30000 non-null  object
 4   question2     30000 non-null  object
 5   is_duplicate  30000 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 1.6+ MB


In [7]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from nltk.corpus import stopwords

# Download stopwords if you haven't already
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def question_preprocessing(question):
  question = question.lower()
  # Remove HTML tags
  question = re.sub(r'<.*?>', '', question)
  question = question.replace('?', '')
  question = question.replace('!', '')
  question = question.replace(',', '')
  question = question.replace('.', '')
  question = question.replace(':', '')
  question = question.replace(';', '')
  question = question.replace('(', '')
  question = question.replace(')', '')
  question = question.replace('[', '')
  question = question.replace(']', '')
  question = re.sub(r"[^a-zA-Z\s]", "", question)
  tokens = word_tokenize(question)

  # Remove stop words
  filtered_tokens = [word for word in tokens if word not in stop_words]

  # Generate trigrams
  trigrams = list(ngrams(filtered_tokens, 3))
  trigram_strings = ['_'.join(gram) for gram in trigrams]

  # Combine unigrams and trigrams
  processed_question = filtered_tokens + trigram_strings

  return processed_question

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
df['question1'] = df['question1'].apply(question_preprocessing)
df['question2'] = df['question2'].apply(question_preprocessing)

In [9]:

df.head()

,id,qid1,qid2,question1,question2,is_duplicate
63416,63416,110364,110365,"[calculate, equivalent, weights, calculate_equ...","[calculate, equivalent, weights, calculate_equ...",1
31741,31741,58499,58500,"[taxes, india, download, soft, copy, pan, card...","[derive, income, tax, file, pan, card, derive_...",0
33081,33081,60824,60825,"[classic, logic, puzzles, classic_logic_puzzles]","[logical, thinking, puzzles, logical_thinking_...",1
184379,184379,281719,281720,"[fourth, year, student, ba, llb, india, one, y...","[second, year, graduation, one, year, left, in...",0
110883,110883,44638,181709,"[evidence, reincarnation]","[concrete, evidence, reincarnation, concrete_e...",1


In [10]:
#till now i have done preprocessing and done tokenization for every question present in both col
#next step is to create vector embedding for these question list  using word2vec



In [12]:
# Combine the tokenized questions from both columns
all_questions = df['question1'].tolist() + df['question2'].tolist()


word2vec_model = Word2Vec(sentences=all_questions, vector_size=300, window=5, min_count=1, workers=4)


word2vec_model.save("word2vec_questions.model")

In [16]:
def get_question_vector(question_tokens, model):

  vectors = []
  for token in question_tokens:
    if token in model.wv:
      vectors.append(model.wv[token])
  if len(vectors) == 0:
    # Return a zero vector if none of the tokens are in the vocabulary
    return np.zeros(model.vector_size)
  else:
    return np.mean(vectors, axis=0)

In [23]:
# Create new features from question vectors
df['vector_diff'] = df.apply(lambda row: np.abs(row['question1_vector'] - row['question2_vector']), axis=1)
df['vector_mul'] = df.apply(lambda row: row['question1_vector'] * row['question2_vector'], axis=1)

# Add features based on question lengths and overlap
df['q1_len'] = df['question1'].apply(lambda x: len(x))
df['q2_len'] = df['question2'].apply(lambda x: len(x))
df['len_diff'] = np.abs(df['q1_len'] - df['q2_len'])

def word_overlap(row):
  q1_words = set(row['question1'])
  q2_words = set(row['question2'])
  return len(q1_words.intersection(q2_words))

df['word_overlap'] = df.apply(word_overlap, axis=1)

# Prepare data for modeling
X = np.concatenate([np.vstack(df['question1_vector'].values),
                    np.vstack(df['question2_vector'].values),
                    np.vstack(df['vector_diff'].values),
                    np.vstack(df['vector_mul'].values),
                    df[['q1_len', 'q2_len', 'len_diff', 'word_overlap']].values], axis=1)

y = df['is_duplicate']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply StandardScaler to the numerical features (the last 4 columns)
scaler = StandardScaler()
X_train[:, -4:] = scaler.fit_transform(X_train[:, -4:])
X_test[:, -4:] = scaler.transform(X_test[:, -4:])


print("Shape of features (X):", X.shape)
print("Shape of target (y):", y.shape)
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Shape of features (X): (30000, 1204)
Shape of target (y): (30000,)
Shape of X_train: (24000, 1204)
Shape of X_test: (6000, 1204)
Shape of y_train: (24000,)
Shape of y_test: (6000,)


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 30000 entries, 63416 to 300323
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   id                30000 non-null  int64 
 1   qid1              30000 non-null  int64 
 2   qid2              30000 non-null  int64 
 3   question1         30000 non-null  object
 4   question2         30000 non-null  object
 5   is_duplicate      30000 non-null  int64 
 6   question1_vector  30000 non-null  object
 7   question2_vector  30000 non-null  object
 8   vector_diff       30000 non-null  object
 9   vector_mul        30000 non-null  object
 10  q1_len            30000 non-null  int64 
 11  q2_len            30000 non-null  int64 
 12  len_diff          30000 non-null  int64 
 13  word_overlap      30000 non-null  int64 
dtypes: int64(8), object(6)
memory usage: 3.4+ MB


In [25]:
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [26]:

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)


loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")


y_pred = (model.predict(X_test) > 0.5).astype("int32")
print(classification_report(y_test, y_pred))

Epoch 1/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.6807 - loss: 0.5672 - val_accuracy: 0.7181 - val_loss: 0.4949
Epoch 2/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7078 - loss: 0.5066 - val_accuracy: 0.7104 - val_loss: 0.4998
Epoch 3/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7177 - loss: 0.4972 - val_accuracy: 0.7183 - val_loss: 0.4952
Epoch 4/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7225 - loss: 0.4923 - val_accuracy: 0.7333 - val_loss: 0.4853
Epoch 5/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7222 - loss: 0.4835 - val_accuracy: 0.7310 - val_loss: 0.4846
Epoch 6/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7310 - loss: 0.4799 - val_accuracy: 0.7335 - val_loss: 0.4811
Epoch 7/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7261 - loss: 0.4836 - val_accuracy: 0.7275 - val_loss: 0.4841
Epoch 8/30
600/600 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7359 - loss: 0.4734 - val_accuracy: 0.